In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
def parse_rating(review_soup):
    value = review_soup.find('span', itemprop='reviewRating').find('meta', itemprop="ratingValue").get('content')
    maxValue = review_soup.find('span', itemprop='reviewRating').find('meta', itemprop="bestRating").get('content')
    
    return float(value), float(maxValue)

In [13]:
def parse_title(review_soup):
    title = review_soup.find('div', class_='review_item_header_content').text
    title = title.strip()[1:-1]
    
    return title

In [14]:
def pase_pos_and_neg(review_soup):
    pos_text, neg_text = None, None 
    
    review_pos = review_soup.find('p', class_='review_pos')
    if review_pos is not None:
        pos_text = review_pos.text.strip()
    
    review_neg = review_soup.find('p', class_='review_neg')
    if review_neg is not None:
        neg_text = review_neg.text.strip()
        
    
    return pos_text, neg_text

In [2]:
def parse_reviews_from_url(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    reviews = soup.findAll("li", class_ = "review_item")


    
    ratingValues, maxRatingValues, titles, pos_texts, neg_texts = [], [], [], [], []
    for review in reviews:
        review_soup = BeautifulSoup(str(review), 'html.parser')
        try:
            ratingValue, bestRating = parse_rating(review_soup)
            title = parse_title(review_soup)
            pos_text, neg_text = pase_pos_and_neg(review_soup)
            
            ratingValues.append(ratingValue)
            maxRatingValues.append(bestRating)
            titles.append(title)
            
            pos_texts.append(pos_text)
            neg_texts.append(neg_text)
                        
        except:
            print('error')
            continue
            
    
    return pd.DataFrame({'title' : titles, 
                         'pos_text' : pos_texts, 
                         'neg_text' : neg_texts, 
                         'ratingValue' : ratingValues, 
                         'bestRating' : maxRatingValues})

In [3]:
def parse_reviews_from_urls(urls):
    frames = []
    for url in urls:
        df = parse_reviews_from_url(url)
        frames.append(df)
    
    return pd.concat(frames)

In [42]:
urls = [
#Blum hotel
    'https://www.booking.com/reviews/ua/hotel/blum.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/blum.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    
#Kryva Lypa 
    'https://www.booking.com/reviews/ua/hotel/kryva-lypa.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/kryva-lypa.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/kryva-lypa.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=3;r_lang=uk;rows=75&',
    
#Ibis style
    'https://www.booking.com/reviews/ua/hotel/ibis-styles-lviv-center.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/ibis-styles-lviv-center.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',

#Central Hotel
    'https://www.booking.com/reviews/ua/hotel/central-mini.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/central-mini.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    
#Atlas de luxe
    'https://www.booking.com/reviews/ua/hotel/atlas-deluxe.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/atlas-deluxe.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    
#Modern Art Hotel
    'https://www.booking.com/reviews/ua/hotel/modern-art.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/modern-art.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    
#Panska Gora
    'https://www.booking.com/reviews/ua/hotel/panska-gora.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/panska-gora.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/panska-gora.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=3;r_lang=uk;rows=75&',
    
#Gruner Lviv
    'https://www.booking.com/reviews/ua/hotel/gruner.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    
#FERENC Hotel
    'https://www.booking.com/reviews/ua/hotel/ferenc-amp-restaurant.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total&hp_nav=0&order=featuredreviews&page=1&r_lang=uk&rows=75&',
    
#Opera Center Hotel
    'https://www.booking.com/reviews/ua/hotel/opera-center-apart.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    
#Dream Hostle Lviv
    'https://www.booking.com/reviews/ua/hotel/dream-hostel-lviv.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/dream-hostel-lviv.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/dream-hostel-lviv.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=3;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/dream-hostel-lviv.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=4;r_lang=uk;rows=75&',
    
#FERENC Hotel
    'https://www.booking.com/reviews/ua/hotel/ferenc-amp-restaurant.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    
#TAURUS
    'https://www.booking.com/reviews/ua/hotel/taurus.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/taurus.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/taurus.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=3;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/taurus.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=4;r_lang=uk;rows=75&',
    
#Sun Hostel
    'https://www.booking.com/reviews/ua/hotel/hostel-sun.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/hostel-sun.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/hostel-sun.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=3;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/hostel-sun.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=4;r_lang=uk;rows=75&',
    
#Three crowns
    'https://www.booking.com/reviews/ua/hotel/three-crowns.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',

#10-Rooms 
    'https://www.booking.com/reviews/ua/hotel/10-rooms.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=1;r_lang=uk;rows=75&',
    'https://www.booking.com/reviews/ua/hotel/10-rooms.uk.html?label=gen173nr-1DCA0o6QFCCmhvc3RlbC1zdW5IKVgEaOkBiAEBmAEpuAEHyAEN2AED6AEB-AECiAIBqAIDuAKQgdPrBcACAQ;sid=3ead4aec4349c417ba3530e6eaa03bff;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page=2;r_lang=uk;rows=75&'
]

In [43]:
df = parse_reviews_from_urls(urls)

error
error
error
error


In [44]:
df

,title,pos_text,neg_text,ratingValue,bestRating
0,наступної мандрівки до Львова оберемо цей готель.,Відмінне розташування готелю! Дуже близько до ...,"один маленький мінус, але нам це не завдало не...",10.0,10.0
1,"Сніданок - великі порціїї, смачно!","Сніданок - великі порціїї, смачно! 1 страва бе...",Сам готель усередині нагадує гуртожиток чи хос...,6.3,10.0
2,"Все чудово! За нагоди, обов'язково знову завіт...",Дуже чудовий і затишний отель! Дуже зручне роз...,"Сантехніку (труби , стоки) необхідно перевірит...",9.6,10.0
3,"Отель знаходиться у центрі міста, але при цьом...","Отель знаходиться у центрі міста, але при цьом...",None,10.0,10.0
4,"класне розташування, такий собі острівець тиші...","класне розташування, такий собі острівець тиші...",начебто новий ремонт - а велика тріщина над дв...,7.5,10.0
5,"Гарне місце для туристів або відрядження, але ...","Ідеальний номер, як для тризіркового готелю - ...",Дуже круті сходи. Складно знайти поруч місце д...,9.2,10.0
6,"Дуже душно, номер з вікном на стелі, дотягнути...",None,"Дуже душно, номер з вікном на стелі, дотягнути...",4.6,10.0
7,- Чудове розташування у тихій вуличці у самому...,- Чудове розташування у тихій вуличці у самому...,- Дуже круті сходи між поверхами і відсутність...,7.1,10.0
8,"Зручне розташування, в номері чисто, є все нео...","Зручне розташування, в номері чисто, є все нео...",None,9.2,10.0
9,"Дуже приємний персонал, тиша навкруги і дуже б...","Дуже приємний персонал, тиша навкруги і дуже б...",Не було полиці в ванній кімнаті. Цього дуже не...,10.0,10.0
